Load DataSet

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
!nvidia-smi

Thu Oct 26 07:20:53 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   49C    P2              30W / 170W |   5587MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
import tensorflow as tf
physycal_devices=tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physycal_devices[0], True)
tf.config.experimental.set_memory_growth(physycal_devices[1], True)
tf.config.experimental.set_synchronous_execution(False)
print(tf.__version__)
# 检查 TensorFlow 是否正确识别了 GPU
#print("GPU is available:", tf.test.is_gpu_available())
# 打印 CUDA 版本和 cuDNN 版本信息
print("CUDA version:", tf.test.is_built_with_cuda())
#print("cuDNN version:", tf.test.is_built_with_cudnn())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for i, gpu in enumerate(gpus):
        print(f"GPU {i+1}: {gpu.name}, device_type={gpu.device_type},GPU Memory:{tf.config.experimental.get_memory_growth(gpu)}")
else:
    print("No GPU devices found.")
# 设置只可见第二个 GPU 设备
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


RuntimeError: Physical devices cannot be modified after being initialized

In [5]:
class DataGenerator_for_two_branch(Sequence):

    def __init__(self, img_paths, label_paths, img_size, batch_size):
        self.img_paths = img_paths
        self.label_paths = label_paths
        self.img_size = img_size
        self.batch_size = batch_size

    def __getitem__(self, index):
        img_start_idx, img_end_idx = self.index_batch_to_image(index)

        img_path_list = self.img_paths[img_start_idx:img_end_idx]
        label_path_list = self.label_paths[img_start_idx:img_end_idx]

        X = [self.load_and_preprocess_image(img_path) for img_path in img_path_list]
        Y = [self.load_and_preprocess_image(label_path) for label_path in label_path_list]

        return np.array(X), np.array(Y)

    def __len__(self):
        return len(self.img_paths) // self.batch_size

    def index_batch_to_image(self, batch_index):
        image_index_start = batch_index * self.batch_size
        image_index_end = (batch_index + 1) * self.batch_size

        return image_index_start, image_index_end

    def load_and_preprocess_image(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, self.img_size)
        img = img / 255.0
        img = np.expand_dims(img, axis=-1)
        return img


def get_paths_with_prefix(img_folder, label_folder, prefix_to_keep):
    img_paths = sorted([os.path.join(img_folder, file) for file in os.listdir(img_folder) if file.startswith(prefix_to_keep)])
    label_paths = sorted([os.path.join(label_folder, file) for file in os.listdir(label_folder) if file.startswith(prefix_to_keep)])
    return img_paths, label_paths

# 设置路径
img_folder = "train"
label_folder = "train_mask"
val_folder = "val"
valmask_folder = "val_mask"

input_shape = 256
batch_size = 8
img_size = (input_shape, input_shape)

In [6]:
def show_history(history, validation : bool = False):
    if validation:
        # Loss
        fig, axes = plt.subplots(figsize= (20,5))
        # Train
        axes.plot(history.epoch, history.history['loss'], color= 'r',  label = 'Train')
        axes.plot(history.epoch, history.history['val_loss'], color = 'b', label = 'Val')
        axes.set_xlabel('Epoch')
        axes.set_ylabel('Loss')
        axes.legend()
        # Acc
        fig, axes = plt.subplots(figsize= (20,5))
        # Train
        axes.plot(history.epoch, history.history['accuracy'], color= 'r',  label = 'Train')
        axes.plot(history.epoch, history.history['val_accuracy'], color = 'b', label = 'Val')
        axes.set_xlabel('Epoch')
        axes.set_ylabel('accuracy')
        axes.legend()
        # Mean Iou
        fig, axes = plt.subplots(figsize= (20,5))
        # Train
        #axes.plot(history.epoch, history.history['mean_iou'], color= 'r',  label = 'Train')
        #axes.plot(history.epoch, history.history['val_mean_iou'], color = 'b', label = 'Val')
        #axes.set_xlabel('Epoch')
        #axes.set_ylabel('MeanIoU')
        #axes.legend()
    else:
        fig, axes = plt.subplots(1,4, figsize= (20,5))
        # loss
        axes[0].plot(history.epoch, history.history['loss'])
        axes[0].set_title('Train')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        # Acc
        axes[1].plot(history.epoch, history.history['accuracy'])
        axes[1].set_title('Train')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Acc')
        # Mean Iou
        #axes[2].plot(history.epoch, history.history['mean_iou'])
        #axes[2].set_title('Train')
        #axes[2].set_xlabel('Epoch')
        #axes[2].set_ylabel('MeanIoU') 

In [7]:
import tensorflow as tf

class PReLU(tf.keras.layers.Layer):
    def __init__(self, num_parameters=1, init=0.25):
        super(PReLU, self).__init__()
        self.num_parameters = num_parameters
        self.alpha = self.add_weight(shape=(num_parameters,),
                                     initializer=tf.keras.initializers.Constant(init),
                                     trainable=True)

    def call(self, inputs):
        pos = tf.nn.relu(inputs)
        neg = self.alpha * (inputs - tf.abs(inputs)) * 0.5
        return pos + neg


Original Model (UNET)

In [8]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras import Input, Model

def unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='PReLU', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='PReLU', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='PReLU', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='PReLU', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='PReLU', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='PReLU', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='PReLU', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='PReLU', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='PReLU', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='PReLU', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2D(512, 2, activation='PReLU', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='PReLU', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='PReLU', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='PReLU', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='PReLU', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='PReLU', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='PReLU', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='PReLU', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='PReLU', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='PReLU', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='PReLU', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='PReLU', padding='same')(conv9)
    conv9 = Conv2D(2, 3, activation = 'PReLU', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    
    return model

input_shape = (256, 256, 1)
model = unet(input_shape)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 256, 256, 64) 4194944     input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 4231232     conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_25[0][0]                  
____________________________________________________________________________________________

顯示載入資料集與資料及數量

In [9]:
# 修改show_random_images函数，使其接受img_paths和label_paths作为参数
def show_random_images(img_paths, label_paths, num_samples=5):
    # 从图像路径列表中随机选择 num_samples 个索引
    random_indices = random.sample(range(len(img_paths)), num_samples)

    # 创建一个 (num_samples, 2) 的子图布局
    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 10))

    # 遍历随机选择的索引
    for i, index in enumerate(random_indices):
        img_path = img_paths[index]
        label_path = label_paths[index]

        # 加载原图像和对应的 mask 图像
        img = cv2.imread(img_path)
        label = cv2.imread(label_path)

        # 显示原图像
        axes[i, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[i, 0].set_title('Original Image')

        # 显示 mask 图像
        axes[i, 1].imshow(cv2.cvtColor(label, cv2.COLOR_BGR2RGB))
        axes[i, 1].set_title('Mask Image')


    # 确保子图之间不重叠
    plt.tight_layout()
    plt.show()

def Print(train_gen,val_gen):
    num_images = len(train_gen.img_paths)
    num_images1 = len(train_gen.label_paths)
    num_images2 = len(val_gen.img_paths)
    num_images3 = len(val_gen.label_paths)
    print("圖像数量为:", num_images,num_images1,num_images2,num_images3)


資料擴增函示

In [10]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_data(input_image_folder, input_label_folder, output_image_folder, output_label_folder, num_augmentations=30):
    # 创建图像数据生成器
    datagen = ImageDataGenerator(
        zca_whitening=False,
        rotation_range=0.2,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        zoom_range=0.05,
        horizontal_flip=True,
        fill_mode="reflect"
    )

    # 检查输出文件夹是否存在，如果不存在则创建它
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)

    if not os.path.exists(output_label_folder):
        os.makedirs(output_label_folder)

    image_filenames = [filename for filename in os.listdir(input_image_folder) if filename.endswith(".png")]
    label_filenames = [filename for filename in os.listdir(input_label_folder) if filename.endswith(".png")]

    # 存储所有数据增强后的图像和标签
    augmented_images = []
    augmented_labels = []
    augmented_image_paths = []
    augmented_label_paths = []

    for i, (input_image_filename, input_label_filename) in enumerate(zip(image_filenames, label_filenames)):
        img_path = os.path.join(input_image_folder, input_image_filename)
        label_path = os.path.join(input_label_folder, input_label_filename)

        img = cv2.imread(img_path)
        label = cv2.imread(label_path)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)

        for j in range(num_augmentations):
            # 使用不同的随机种子对图像和标签进行数据增强
            seed = np.random.randint(0, 1000)  # 选择随机种子
            img_generator = datagen.flow(np.array([img]), batch_size=1, seed=seed)
            label_generator = datagen.flow(np.array([label]), batch_size=1, seed=seed)

            img_batch = img_generator.next()
            label_batch = label_generator.next()

            img_batch = cv2.cvtColor(img_batch[0], cv2.COLOR_RGB2BGR)
            label_batch = cv2.cvtColor(label_batch[0], cv2.COLOR_RGB2BGR)

            augmented_img_filename = os.path.join(output_image_folder, f"img_{i}aug{j}_{input_image_filename}")
            augmented_label_filename = os.path.join(output_label_folder, f"label_{i}aug{j}_{input_label_filename}")

            cv2.imwrite(augmented_img_filename, img_batch)
            cv2.imwrite(augmented_label_filename, label_batch)

            augmented_images.append(img_batch)
            augmented_labels.append(label_batch)
            augmented_image_paths.append(augmented_img_filename)
            augmented_label_paths.append(augmented_label_filename)

    return augmented_images, augmented_labels, augmented_image_paths, augmented_label_paths


def display_augmented_data(augmented_images, augmented_labels, num_samples=9):
    # 随机选择 num_samples 张图像进行显示
    random_indices = random.sample(range(len(augmented_images)), num_samples)
    plt.figure(figsize=(12, 6))
    for i, idx in enumerate(random_indices):
        img_batch = augmented_images[idx]
        label_batch = augmented_labels[idx]

        img_batch = img_batch / 255.0
        label_batch = label_batch / 255.0

        plt.subplot(3, 6, i * 2 + 1)  # 输出图像
        plt.imshow(img_batch)
        plt.title(f"Image {i + 1}")
        plt.axis('off')

        plt.subplot(3, 6, i * 2 + 2)  # 输出标签
        plt.imshow(label_batch)
        plt.title(f"Label {i + 1}")
        plt.axis('off')

    plt.show()
#資料擴增後的存檔路徑

output_image_folder = "test_img"
output_label_folder = "test_label"


載入原圖片資料夾 與 結合資料擴增的函示

In [8]:
# 修改 create_data_generator 函数
def create_data_generator(img_folder, label_folder, val_folder, valmask_folder, prefix_to_keep, img_size, batch_size):
    train_img_paths, train_label_paths = get_paths_with_prefix(img_folder, label_folder, prefix_to_keep)
    val_img_paths, val_label_paths = get_paths_with_prefix(val_folder, valmask_folder, prefix_to_keep)
    #print(val_img_paths)
    # 数据增强
    augmented_images, augmented_labels,aug_image_paths,aug_label_paths = augment_data(img_folder, label_folder, output_image_folder, output_label_folder)
    display_augmented_data(augmented_images, augmented_labels)
    #merged_images, merged_labels = combine_images_and_labels(augmented_images, augmented_labels, img_folder, label_folder)
   
    #print("images Shape:",np.array(merged_images).shape)      
    #print("images Shape:",np.array(merged_labels).shape)
    
    merged_images=train_img_paths+aug_image_paths
    merged_labels=train_label_paths+aug_label_paths
    
    
    # 创建数据生成器
    train_generator = DataGenerator_for_two_branch(
        img_paths=merged_images,
        label_paths=merged_labels,
        img_size=img_size,
        batch_size=batch_size
    )

    val_generator = DataGenerator_for_two_branch(
        img_paths=val_img_paths,
        label_paths=val_label_paths,
        img_size=img_size,
        batch_size=batch_size
    )

    return train_generator, val_generator

#prefix_to_keep=""
#train_generator, val_generator = create_data_generator(img_folder, label_folder, val_folder, valmask_folder, prefix_to_keep, img_size, batch_size)


In [11]:
def create_data_generator(img_folder, label_folder, val_folder, valmask_folder, prefix_to_keep, img_size, batch_size):
    train_img_paths, train_label_paths = get_paths_with_prefix(img_folder, label_folder, prefix_to_keep)
    val_img_paths, val_label_paths = get_paths_with_prefix(val_folder, valmask_folder, prefix_to_keep)

    
    
    # 创建数据生成器
    train_generator = DataGenerator_for_two_branch(
        img_paths=train_img_paths,
        label_paths=train_label_paths,
        img_size=img_size,
        batch_size=batch_size
    )

    val_generator = DataGenerator_for_two_branch(
        img_paths=val_img_paths,
        label_paths=val_label_paths,
        img_size=img_size,
        batch_size=batch_size
    )

    return train_generator, val_generator

Train

In [12]:
#训练模型
def train_model(train_generator, val_generator, model, checkpoint_filepath, epochs):
# 创建 Adam 优化器并设置学习率
    custom_optimizer = Adam(learning_rate=0.00001)
# 使用自定义的优化器来编译模型
    model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=["accuracy"],run_eagerly= True)
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                          save_best_only=True,  
                                          monitor='loss',  
                                          mode='min',  
                                          verbose=1)  
     
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[checkpoint_callback]
        )
    show_history(history, True)

In [ ]:
import os
def delete_all_files_in_folder(folder_path):
    # 检查文件夹是否存在
    if not os.path.exists(folder_path):
        print(f"Folder does not exist: {folder_path}")
        return

    # 遍历文件夹内的所有文件并删除它们
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {str(e)}")

# 调用函数来删除指定文件夹内的所有文件
delete_all_files_in_folder("test_img")
delete_all_files_in_folder("test_label")


In [13]:
prefix = "CFD"
train_gen, val_gen= create_data_generator(img_folder, label_folder, val_folder, valmask_folder, prefix, img_size, batch_size)
Print(train_gen,val_gen)
#show_random_images(train_gen.img_paths, train_gen.label_paths, num_samples=2)


圖像数量为: 70 70 25 25


In [10]:
prefix="CFD"
checkpoint_filepath = f'{prefix}nonaug_model.h5'
train_model(train_gen,val_gen , model, checkpoint_filepath, epochs=30)

Epoch 1/30
1/8 [==>...........................] - ETA: 1:56 - loss: 0.6247 - accuracy: 0.7488

ResourceExhaustedError: OOM when allocating tensor with shape[8,64,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:FusedBatchNormV3]